## Replicate QC

This notebook takes a merged sampleset, ie all GT arrays from a set that have been merged by the `combine-zarr-callset` pipeline, and computes pairwise distances between them.

Each contig is handled separately, and the dimensions of the resulting outputs are: contigs x npairs.

Three arrays are written, one with euclidean distance, one with cityblock distance and one with the number of comparable (ie called) sites.

NB: We restrict to bialleleic positions in phase 2. 

NB: Efficiency could be improved markedly by chunking the genotypes with (X, 1, 2), to make it more efficient at reading in the relevant data. 
(I'm currently recreating on the cluster)

In [1]:
import zarr
import allel
import pandas as pd

In [2]:
!pip install dask-distance
import dask_distance as dist
import os

You are using pip version 19.0, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
import dask.array as da
import numpy as np

In [4]:
sampleset = "AG1000G-UG"

In [5]:
calldata = zarr.open_group("/gcs/observatory/callset.zarr", mode="r")

In [6]:
df = pd.read_csv("/gcs/observatory/manifest")

In [7]:
# assume this is ok for now. Normally use the manifest
samples = df["sample_name"].tolist()

In [8]:
from dask_kubernetes import KubeCluster
cluster = KubeCluster(n_workers=40)
cluster

In [9]:
from dask.distributed import Client, progress
client = Client(cluster)

In [10]:
client

Client Scheduler: tcp://10.8.71.16:34861 Dashboard: /user/nicholasharding/proxy/8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [11]:
len(samples)

361

In [12]:
phase2_callset = zarr.open_group("/gcs/phase2/AR1/variation/main/zarr2/ag1000g.phase2.ar1")
called_sites = zarr.open_group("/gcs/observatory/ag.allsites.nonN.zarr.zip", mode="r")

In [13]:
# find biallelic sites
def find_phase2_bialleleic_sites(chrom):

    g = allel.GenotypeDaskArray(phase2_callset[chrom]["calldata"]["genotype"])
    biallelic = (g.max(axis=[1,2]) <= 1).compute()
                 
    d = {}
    for x in "POS", "REF", "ALT":
        v = phase2_callset[chrom]["variants"][x]
        dav = da.from_zarr(v, chunksize=v.chunks)
        d[x] = da.compress(biallelic, dav, axis=0)
        
    return d["POS"], d["ALT"], d["REF"]

In [14]:
config = {"pwd_contigs": ["3L", "3R", "2L", "2R", "X"]}

In [15]:
import sys
sys.setrecursionlimit(100000)

In [16]:
from itertools import combinations

In [17]:
len(samples)**2

130321

In [18]:
pairs = list(combinations(range(len(samples)), 2))
npairs = len(pairs)
dist_e = np.zeros((len(config["pwd_contigs"]), npairs))
dist_c = np.zeros((len(config["pwd_contigs"]), npairs))
numb_s = np.zeros((len(config["pwd_contigs"]), npairs))

In [ ]:
for cix, contig in enumerate(config["pwd_contigs"]):

    sites_pos = allel.SortedIndex(called_sites[contig]["variants/POS"])
    bial_pos, bial_alt, bial_ref = find_phase2_bialleleic_sites(contig)
    loc = sites_pos.locate_keys(bial_pos)
    
    alleles=da.hstack((bial_ref.reshape((-1, 1)), bial_alt))

    # reduce to biallelic sites all samples still
    print("compressing")
    gt_a = allel.GenotypeDaskArray(calldata[contig]["calldata/GT"]).compress(loc)

    mapping = allel.create_allele_mapping(
        ref=np.compress(loc, called_sites[contig]["variants/REF"]),
        alt=np.compress(loc, called_sites[contig]["variants/ALT"]),
        alleles=alleles)
    
    print("mapped ok")
    remap = gt_a.map_alleles(mapping)
    count_alts = remap.to_n_alt()
    is_called = remap.is_called()
    
    for i, j in pairs:
        ix = allel.condensed_coords(i, j, len(samples))
        nmloc = (is_called[:, i] & is_called[:, j]).compute()

        x1 = da.compress(nmloc, count_alts[:, i], axis=0)
        x2 = da.compress(nmloc, count_alts[:, j], axis=0)
            
        dist_e[cix, ix] = dist.euclidean(x1, x2)
        dist_c[cix, ix] = dist.cityblock(x1, x2)
        numb_s[cix, ix] = x1.shape[0]
        
    break

compressing
mapped ok


In [ ]:
np.savez_compressed(
    'pwdist_{sampleset}'.format(sampleset=sampleset), 
    euclidean=dist_e, 
    cityblock=dist_c, 
    number_sites=numb_s)